# NOAA Tides & Currents
- Sewells Point: https://tidesandcurrents.noaa.gov/inventory.html?id=8638610
- https://pypi.org/project/noaa-coops/
- product: https://api.tidesandcurrents.noaa.gov/api/prod/#products
- datum: https://api.tidesandcurrents.noaa.gov/api/prod/#datum:~:text=for%20the%20station-,Datum,-The%20datum%20can
- Output columns ['v', 's', 'f', 'q'] https://api.tidesandcurrents.noaa.gov/api/prod/responseHelp.html

In [1]:
# !pip install noaa_coops
from noaa_coops import Station
import pandas as pd
from pprint import pprint

# Sewells Point station
station_id = "8638610"
station = Station(station_id)
pprint(station.data_inventory)

{'Air Temperature': {'end_date': '2025-04-18 02:30',
                     'start_date': '1996-01-11 06:00'},
 'Barometric Pressure': {'end_date': '2025-04-18 02:30',
                         'start_date': '1996-01-11 06:00'},
 'Preliminary 6-Minute Water Level': {'end_date': '2025-04-18 02:30',
                                      'start_date': '2001-01-01 00:00'},
 'Rain Fall': {'end_date': '2002-02-08 20:00',
               'start_date': '1996-01-11 06:00'},
 'Verified 6-Minute Water Level': {'end_date': '2025-03-31 23:54',
                                   'start_date': '1995-12-12 21:48'},
 'Verified High/Low Water Level': {'end_date': '2025-03-31 23:54',
                                   'start_date': '1979-08-01 07:12'},
 'Verified Hourly Height Water Level': {'end_date': '2025-03-31 23:00',
                                        'start_date': '1927-07-22 05:00'},
 'Verified Monthly Mean Water Level': {'end_date': '2025-03-31 23:54',
                                       'st

In [2]:
 # pprint(list(station.metadata.items()))

In [3]:
data = station.get_data(
    begin_date='20150101',
    end_date='20160101',
    product='water_level',
    datum='MHHW', # Mean Higher High Water
    units='metric',
    time_zone='lst', # gmt, lst_ldt
)

In [4]:
display(data.head()) # 6 minute intervals

,v,s,f,q
t,,,,
2015-01-01 00:00:00,-0.855,0.002,"0,0,0,0",v
2015-01-01 00:06:00,-0.849,0.002,"0,0,0,0",v
2015-01-01 00:12:00,-0.843,0.002,"0,0,0,0",v
2015-01-01 00:18:00,-0.835,0.003,"0,0,0,0",v
2015-01-01 00:24:00,-0.827,0.002,"0,0,0,0",v


# NWIS
### Stations near LaRC
- Not all stations have useful parameters
- Brick Kiln Creek Below Route 172 Near Poquoson, VA - 0167862550:
    - https://waterdata.usgs.gov/monitoring-location/0167862550/#dataTypeId=continuous-62620-0&period=P7D&showMedian=true
- Hampton River at Route 351 at Hampton, VA - 0204289985:
    - https://waterdata.usgs.gov/monitoring-location/0204289985/#dataTypeId=continuous-62620-0&period=P7D&showMedian=true
- James River at Fort Monroe, VA - 0204289994:
    - https://waterdata.usgs.gov/monitoring-location/0204289994/#dataTypeId=continuous-62620-0&period=P7D&showMedian=true
- Newmarket Creek at Mercury Blvd at Hampton, VA - 0167892964:
    - https://waterdata.usgs.gov/monitoring-location/0167892964/#dataTypeId=continuous-62620-0&period=P7D&showMedian=true
- Storm Drain at Coliseum Drive at Hampton, VA - 0167891721:
    - https://waterdata.usgs.gov/monitoring-location/0167891721/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false
- Storm Drain West of Garrett Drive at Hampton, VA - 0167889257:
    - https://waterdata.usgs.gov/monitoring-location/0167889257/#dataTypeId=continuous-00065-0&period=P7D&showMedian=false
- 59D 25 Sow 213 - 365756076251201:
    - https://waterdata.usgs.gov/monitoring-location/365756076251201/#dataTypeId=continuous-72019-0&period=P7D&showMedian=true
### parameters
- https://doi-usgs.github.io/dataretrieval-python/examples/index.html
- https://help.waterdata.usgs.gov/parameter_cd?group_cd=PHY
    - 00061, 00035, 00036, 00045, ...
 


In [5]:
import pandas as pd
from geopy.distance import geodesic
import dataretrieval.nwis as nwis

coord = (37.0862, -76.3809) # LaRC
param_codes = {
    '00061': 'Discharge',
    '00035': 'Wind Speed',
    '00036': 'Wind Direction',
    '00045': 'Precipitation'
}

results = []

for code, desc in param_codes.items():
    site_data, _ = nwis.what_sites(stateCd='VA', parameterCd=code)
    site_data = site_data.dropna(subset=['dec_lat_va', 'dec_long_va'])
    site_data['distance_km'] = site_data.apply(
        lambda row: geodesic(coord, (row['dec_lat_va'], row['dec_long_va'])).km, axis=1)
    nearest = site_data.sort_values('distance_km').head()
    nearest['param_code'] = code
    nearest['param_desc'] = desc
    results.append(nearest[['site_no', 'station_nm', 'dec_lat_va', 'dec_long_va', 'distance_km', 'param_code', 'param_desc']])

final_df = pd.concat(results).reset_index(drop=True)
display(final_df)


,site_no,station_nm,dec_lat_va,dec_long_va,distance_km,param_code,param_desc
0,0167891721,"STORM DRAIN AT COLISEUM DRIVE AT HAMPTON, VA",37.037694,-76.386111,5.403004,00061,Discharge
1,0167885050,"UNNAMED TRIB TO NEWMARKET CREEK AT HAMPTON, VA",37.050978,-76.442536,6.732417,00061,Discharge
2,0167889257,"STORM DRAIN WEST OF GARRETT DRIVE AT HAMPTON, VA",37.018497,-76.393528,7.597060,00061,Discharge
3,0204279294,"STORM DRAIN AT LAKEWOOD PK DR NR NEWPORT NEWS, VA",37.102550,-76.544431,14.650918,00061,Discharge
4,02042790,"BEAVERDAM CREEK NEAR YORKTOWN, VA",37.202645,-76.520785,17.929204,00061,Discharge
5,365119076174600,"THE HAGUE AT PEDESTRIAN BRIDGE AT NORFOLK, VA",36.855269,-76.296169,26.715536,00035,Wind Speed
6,365123107613412,"ELIZABETH RIVER AT CROWN POINT NORFOLK, VA",36.856537,-76.227772,28.905495,00035,Wind Speed
7,0204288721,EB ELIZABETH RIVER AT RT 165 AT VIRGINIA BEACH...,36.827370,-76.165214,34.555381,00035,Wind Speed
8,0204291317,"THALIA CREEK AT ROUTE 58 AT THALIA, VA",36.843306,-76.124472,35.328055,00035,Wind Speed
9,0204293125,LONDON BRIDGE CREEK AT RT 58 AT VIRGINIA BEACH...,36.841139,-76.057139,39.634868,00035,Wind Speed


In [6]:
# # first import the functions for downloading data from NWIS
# import dataretrieval.nwis as nwis

# # specify the USGS site code for which we want data.
# site = ['0167862550', '0204289985', '0167892964', '0167891721', '0167889257', '365756076251201']


# # get instantaneous values (iv)
# df = nwis.get_record(sites=site, service='iv', start='2017-12-31', end='2018-01-01')

# # get water quality samples (qwdata)
# # df2 = nwis.get_qwdata(sites=site)

# # get basic info about the site
# df3 = nwis.get_record(sites=site, service='site')